to check:
- check schema, maybe change to correct dadta types (could hapend that a prod code is a numb and not a string and inferred as int)

In [0]:
#dbutils.fs.rm('/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv')

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import os

# Settings

In [0]:
first_full_load=False # True only if first historical full load, otherwise leave as Flse for incremental load

### Paths to folders

In [0]:
landing_path='/FileStore/mnt/test_bucket/landing/'
archive_path='/FileStore/mnt/test_bucket/archive/'
silver_path='/FileStore/mnt/test_bucket/silver/'
gold_path='/FileStore/mnt/test_bucket/gold/'

# Product table

In [0]:
table_name = 'products'
table_schema=StructType([StructField('product_code', StringType(), False), StructField('product_name', StringType(), True), StructField('category', StringType(), True), StructField('description', StringType(), True), StructField('gender', StringType(), True), StructField('color', StringType(), True), StructField('size', StringType(), True)])

### Load data

In [0]:
# read all files inside folder
df=(
    spark.read.csv(landing_path + table_name, header=True, schema=table_schema).select('*', '_metadata.*').drop('file_name')
)

In [0]:
display(df)

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000
PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000
PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000
PR0004,Product 4,shoes,sport shoes,female,blue,7,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000


### Data cleaning

In [0]:
# Remove any possible duplicates
df=df.dropDuplicates(subset=['product_code', 'product_name', 'category', 'description', 'gender', 'color', 'size'])

In [0]:
if first_full_load: 
    #df.write.format('delta').option('mergeSchema', 'true').mode('overwrite').saveAsTable('silver.' + table_name + '_silver')

    # add columns for scd 2
    df=df.withColumn("active_status", lit('Y')).withColumn("valid_from", lit("2023-01-01")).withColumn("valid_to", lit("9999-12-31"))

    # create temp view
    df.createOrReplaceTempView('df')

    # insert values
    spark.sql(
        '''
        insert into silver.products_silver (product_code, product_name, category, description, gender, color, size, file_path, file_size, file_modification_time, active_status, valid_from, valid_to)
        select * from df
        '''
    )

In [0]:
# added in order to register table in catalog if usisng new cluster (usually happnes working with community edition)
spark.sql('create database if not exists silver')
spark.sql('''create table if not exists silver.products_silver using delta
          location "/FileStore/mnt/test_bucket/silver/products_silver"''')


Out[4]: DataFrame[]

### SCD 2

In [0]:
join_columns=["product_code"]
main_columns_source=["product_name", "category", "description", "gender", "color", "size"]
main_columns_target=[f"target_{x}" for x in main_columns_source]
all_columns=["ProductId", "product_code", "product_name", "category", "description", "gender", "color", "size", "file_path", "file_size", "file_modification_time", "active_status", "valid_from", "valid_to"]

In [0]:
targetDF=spark.read.format('delta').table('silver.products_silver')
sourceDF=df

##### Step 1 - left join to new data the main table

In [0]:
#display(targetDF)

In [0]:
#display(sourceDF)

In [0]:
text=''
for c in all_columns:
    text+=f'targetDF["{c}"].alias("target_{c}"), '
print(text)

targetDF["product_code"].alias("target_product_code"), targetDF["product_name"].alias("target_product_name"), targetDF["category"].alias("target_category"), targetDF["description"].alias("target_description"), targetDF["gender"].alias("target_gender"), targetDF["color"].alias("target_color"), targetDF["size"].alias("target_size"), targetDF["file_path"].alias("target_file_path"), targetDF["file_size"].alias("target_file_size"), targetDF["file_modification_time"].alias("target_file_modification_time"), targetDF["active_status"].alias("target_active_status"), targetDF["valid_from"].alias("target_valid_from"), targetDF["valid_to"].alias("target_valid_to"), 


In [0]:
## Here all columns including PK
joinDF=(
    sourceDF.join(
        targetDF.where(col("active_status")=='Y'),
        #(sourceDF.botany_order_no==targetDF.botany_order_no) & (sourceDF.line_no==targetDF.line_no) & (targetDF.active_status=='Y'),
        #(sourceDF["product_code"]==targetDF["product_code"])  & (targetDF["active_status"]=='Y'),
        *join_columns,
        'leftouter'
    )
    .select(
        sourceDF['*'],
        targetDF["ProductId"].alias("target_ProductId"),
        targetDF["product_code"].alias("target_product_code"), targetDF["product_name"].alias("target_product_name"), targetDF["category"].alias("target_category"), targetDF["description"].alias("target_description"), targetDF["gender"].alias("target_gender"), targetDF["color"].alias("target_color"), targetDF["size"].alias("target_size")
    )
)

display(joinDF)

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,ProductId,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,active_status,valid_from,valid_to
PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,7,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,8,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
PR0004,Product 4,shoes,sport shoes,female,blue,7,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,3,Product 4,shoes,sport shoes,female,blue,7,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,10,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,9,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31


##### Step 2 - Filter out only changed records

Here only columns i need, not columns that have PK

In [0]:
text1=''
text2=''
for col in main_columns_source:
    text1+=f'col("{col}"), '
    text2+=f'col("target_{col}"), '
print(text1)
print(text2)

In [0]:
filterDF=(
        joinDF.filter(
            xxhash64(col("product_name"), col("category"), col("description"), col("gender"), col("color"), col("size"))
            != xxhash64(col("target_product_name"), col("target_category"), col("target_description"), col("target_gender"), col("target_color"), col("target_size"))
        )
    )

display(filterDF)

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,target_ProductId,target_product_code,target_product_name,target_category,target_description,target_gender,target_color,target_size


##### Step 3 - create merge key

In [0]:
cols=['product_code', 'color', 'size']
#display(sourceDF.withColumn('MERGEKEY', concat(sourceDF['product_code'], sourceDF['product_name'])))
display(sourceDF.withColumn('MERGEKEY', concat(*cols)))

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,MERGEKEY
PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,PR0098black32
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,PR0002white42
PR0004,Product 4,shoes,sport shoes,female,blue,7,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,PR0004blue7
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,PR0001black8
PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,PR0099blueUN


In [0]:
#mergeDF=filterDF.withColumn('MERGEKEY', concat(filterDF.botany_order_no, filterDF.line_no))
mergeDF=filterDF.withColumn('MERGEKEY', concat(*join_columns))

display(mergeDF)

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,target_ProductId,target_product_code,target_product_name,target_category,target_description,target_gender,target_color,target_size,MERGEKEY
PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,null,null,null,null,null,null,null,null,PR0098
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,2,PR0002,Product 2,pants,long pants,female,white,42,PR0002
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,4,PR0001,Product 1,shoes,running shoes,unisex,black,8,PR0001
PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,null,null,null,null,null,null,null,null,PR0099


##### Step 4 - create null merge key for matching records

In [0]:
dummyDF=filterDF.filter('target_ProductId is not null').withColumn('MERGEKEY', lit(None))

display(dummyDF)

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,target_ProductId,target_product_code,target_product_name,target_category,target_description,target_gender,target_color,target_size,MERGEKEY
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,2,PR0002,Product 2,pants,long pants,female,white,42,null
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,4,PR0001,Product 1,shoes,running shoes,unisex,black,8,null


##### Step 5 - combine step 3 and 4

In [0]:
scdDF=mergeDF.union(dummyDF)
scdDF.createOrReplaceTempView('scdDF')
display(scdDF)

product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,target_ProductId,target_product_code,target_product_name,target_category,target_description,target_gender,target_color,target_size,MERGEKEY
PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,null,null,null,null,null,null,null,null,PR0098
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,2,PR0002,Product 2,pants,long pants,female,white,42,PR0002
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,4,PR0001,Product 1,shoes,running shoes,unisex,black,8,PR0001
PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,null,null,null,null,null,null,null,null,PR0099
PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,2,PR0002,Product 2,pants,long pants,female,white,42,null
PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,4,PR0001,Product 1,shoes,running shoes,unisex,black,8,null


##### Step 6 - apply merge statement

In [0]:
%sql
drop table if exists test_merge2;
create table test_merge2 as
select * from silver.products_silver;
select * from test_merge2;

In [0]:
%sql
select * from test_merge2;

In [0]:
text=''
for c in all_columns:
    text+=f'source.{c}, '
text

In [0]:
%sql
merge into silver.products_silver as target
--merge into test_merge2 as target

  using scdDF as source
  on  target.product_code = source.MERGEKEY and target.active_status="Y"

  when matched then update set
    active_status='N',
    valid_to=date_sub(CAST(current_date() as DATE), 1)

  when not matched then insert (product_code, product_name, category, description, gender, color, size, file_path, file_size, file_modification_time, active_status, valid_from, valid_to)
    values(
      source.product_code, 
      source.product_name, 
      source.category, 
      source.description, 
      source.gender, 
      source.color, 
      source.size, 
      source.file_path, 
      source.file_size, 
      source.file_modification_time, 
      'Y', 
      current_date(),
      '9999-12-31')


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
6,2,0,4


In [0]:
%sql
select * from silver.products_silver order by ProductId;

ProductId,product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,active_status,valid_from,valid_to
1,PR0005,Product 5,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
2,PR0002,Product 2,pants,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,N,2023-01-01,2023-11-27
3,PR0004,Product 4,shoes,sport shoes,female,blue,7,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
4,PR0001,Product 1,shoes,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,N,2023-01-01,2023-11-27
5,PR0003,Product 3,underwear,underwear male,male,red,M,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
6,PR0006,Product 6,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
7,PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
8,PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
9,PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
10,PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31


# Assign SK to fact sales

In [0]:
fact_sales=spark.read.csv('/FileStore/mnt/test_bucket/landing/sales/2022_06_09_091745_sales.csv', header=True, inferSchema=True)
display(fact_sales)

order_num,order_ln,date,price,qty,discount,product_code,brand_id,customer_id,cost
1001,1,2022-06-05,50.0,1,0%,PR0001,BR01,99,32.0
1002,1,2022-06-05,30.0,2,0%,PR0002,BR01,null,12.5
1003,1,2022-06-06,25.0,1,0%,PR0003,BR02,null,10.0
1004,1,2022-06-06,35.0,1,0%,PR0001,BR01,150,32.0
1005,1,2022-06-06,25.5,3,10%,PR0004,BR03,null,11.0
1005,2,2022-06-06,16.9,1,0%,PR0005,BR02,null,8.0
1005,3,2022-06-06,27.0,1,0%,PR0001,BR01,null,32.0
1006,1,2022-06-07,95.0,2,5%,PR0004,BR03,215,11.0
1007,1,2022-06-08,16.0,1,0%,PR0006,BR04,99,8.5
1007,2,2022-06-08,22.0,1,0%,PR0002,BR01,99,12.5


In [0]:
fact_sales.createOrReplaceTempView('sales')

In [0]:
%sql
select pr.ProductId, s.product_code, s.order_num, s.order_ln, s.date, s.price, s.qty, s.discount, s.brand_id, s.customer_id, s.cost
from sales s
join silver.products_silver pr
on s.product_code=pr.product_code and pr.active_status != 'N'
-- join silver.brands_silver br
-- on s.brand_id=br.brand_id and br.active_status != 'N'
order by s.product_code;


ProductId,product_code,order_num,order_ln,date,price,qty,discount,brand_id,customer_id,cost
10,PR0001,1005,3,2022-06-06,27.0,1,0%,BR01,null,32.0
10,PR0001,1004,1,2022-06-06,35.0,1,0%,BR01,150,32.0
10,PR0001,1001,1,2022-06-05,50.0,1,0%,BR01,99,32.0
8,PR0002,1007,2,2022-06-08,22.0,1,0%,BR01,99,12.5
8,PR0002,1002,1,2022-06-05,30.0,2,0%,BR01,null,12.5
5,PR0003,1003,1,2022-06-06,25.0,1,0%,BR02,null,10.0
3,PR0004,1006,1,2022-06-07,95.0,2,5%,BR03,215,11.0
3,PR0004,1005,1,2022-06-06,25.5,3,10%,BR03,null,11.0
1,PR0005,1005,2,2022-06-06,16.9,1,0%,BR02,null,8.0
6,PR0006,1007,1,2022-06-08,16.0,1,0%,BR04,99,8.5


In [0]:
%sql
select * from sales;

order_num,order_ln,date,price,qty,discount,product_code,brand_id,customer_id,cost
1001,1,2022-06-05,50.0,1,0%,PR0001,BR01,99,32.0
1002,1,2022-06-05,30.0,2,0%,PR0002,BR01,null,12.5
1003,1,2022-06-06,25.0,1,0%,PR0003,BR02,null,10.0
1004,1,2022-06-06,35.0,1,0%,PR0001,BR01,150,32.0
1005,1,2022-06-06,25.5,3,10%,PR0004,BR03,null,11.0
1005,2,2022-06-06,16.9,1,0%,PR0005,BR02,null,8.0
1005,3,2022-06-06,27.0,1,0%,PR0001,BR01,null,32.0
1006,1,2022-06-07,95.0,2,5%,PR0004,BR03,215,11.0
1007,1,2022-06-08,16.0,1,0%,PR0006,BR04,99,8.5
1007,2,2022-06-08,22.0,1,0%,PR0002,BR01,99,12.5


In [0]:
%sql
select * from silver.products_silver;

ProductId,product_code,product_name,category,description,gender,color,size,file_path,file_size,file_modification_time,active_status,valid_from,valid_to
7,PR0098,Product 98,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
2,PR0002,Product 2,pants,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,N,2023-01-01,2023-11-27
4,PR0001,Product 1,shoes,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,N,2023-01-01,2023-11-27
9,PR0099,Product 99,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
8,PR0002,Product 2,changed,long pants,female,white,42,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
10,PR0001,Product 1,changed,running shoes,unisex,black,8,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products___Copy.csv,350,2023-11-28T20:39:02.000+0000,Y,2023-11-28,9999-12-31
1,PR0005,Product 5,pants,long skinny,female,black,32,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
3,PR0004,Product 4,shoes,sport shoes,female,blue,7,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
5,PR0003,Product 3,underwear,underwear male,male,red,M,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31
6,PR0006,Product 6,accessories,accessories young girls,female,blue,UN,dbfs:/FileStore/mnt/test_bucket/landing/products/2022_06_09_091705_products.csv,398,2023-11-28T20:37:57.000+0000,Y,2023-01-01,9999-12-31


In [0]:
fact_sales2=spark.read.csv('/FileStore/mnt/test_bucket/landing/brands/', header=True, inferSchema=True)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1298894578063971>:1
----> 1 fact_sales2=spark.read.csv('/FileStore/mnt/test_bucket/landing/brands/', header=True, inferSchema=True)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:729, in DataFrameReader.csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumns, maxCharsPerColumn, maxMal

In [0]:
len(dbutils.fs.ls('/FileStore/mnt/test_bucket/landing/sales/'))

Out[19]: 1

In [0]:
dbutil.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

### Move file/s into archive folder

In [0]:

def db_list_files(file_path, file_prefix):
  #file_list = [file.path for file in dbutils.fs.ls(file_path) if os.path.basename(file.path).startswith(file_prefix)]
  file_list = [file.path for file in dbutils.fs.ls(file_path)]
  return file_list

files = db_list_files(f'dbfs:{landing_path}{table_name}', 'foobar')

for file in files:
  #dbutils.fs.cp(file, os.path.join(f'dbfs:{archive_path}{table_name}', os.path.basename(file)))
  dbutils.fs.mv(file, os.path.join(f'dbfs:{archive_path}{table_name}', os.path.basename(file)))


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2025562790395254>:6
      3   file_list = [file.path for file in dbutils.fs.ls(file_path)]
      4   return file_list
----> 6 files = db_list_files(f'dbfs:{landing_path}{table_name}', 'foobar')
      8 for file in files:
      9   #dbutils.fs.cp(file, os.path.join(f'dbfs:{archive_path}{table_name}', os.path.basename(file)))
     10   dbutils.fs.mv(file, os.path.join(f'dbfs:{archive_path}{table_name}', os.path.basename(file)))

NameError: name 'table_name' is not defined

In [0]:
import os

In [0]:
for f in [file.path for file in dbutils.fs.ls(f'dbfs:{landing_path}products')]:
    dbutils.fs.mv(f, os.path.join(f'dbfs:{archive_path}products', os.path.basename(f)))

In [0]:
for f in dbutils.fs.ls(f'dbfs:{landing_path}products'):
    dbutils.fs.mv(f.path, os.path.join(f'dbfs:{archive_path}products', os.path.basename(f.path)))